# Chapter 7 Gaussian elimination

## Echelon Form
### Definition 7.1.1
An m × n matrix A is in echelon form if it satisfies the following condition: for any row, if that row’s first nonzero entry is in position k then every previous row’s first nonzero entry is in some position less than k.

A generalization of triangular form.

## Echelon Conversion
Lemma 7.1.2 If a matrix is in echelon form, the nonzero rows form a basis for the row space.

Prove by induction using the grow algorithm.

### Row addition preserves row space.
Use row-addition matrix to perform the echelon conversion, prove it would preserve the row space.

### Computational issues
In exact float arithmatics would cause issues, in practice, select the largest row (for division)

### Lemma 7.1.3: For matrices A and N, Row NA ⊆ Row A.
### Corollary 7.1.4: For matrices A and M, if M is invertible then Row MA = Row A.

Intuition, an invertible transformation is one-to-one; and preserves vector space of Row A.

### Proposition 7.3.1: For any matrix A, there is an invertible matrix M such that MA is in echelon form.


## Tracking the Row Addition Matrix to find Linear Solution
Starting with I, track the row addition performed on A
We could derive M*A = Echelon Form

## Theorem 7.6.1 (Prime Factorization Theorem): 
For every positive integer N, there is a unique bag of primes whose product is N.

In [2]:
# My attempt
def convert_in_echelon(rowlist, col_label_list):
    def find_pivot(row_index, col_label_list, k):
        for c in col_label_list[prev_k+1:]:
            for r in row_index:
                if rowlist[r][c] != 0:
                    return r, c

        # NOTE: all the rows are zeros, pick the last column as k
        return row_index[0], col_label_list[-1]

    def _convert_in_echelon(remaining_rows, prev_k):
        """Convert rowlist to echelon with previous row's nonzero entry at prev_k."""
        if not remaining_rows:
            return []
        assert all([rowlist[r][c] == 0
                    for c in col_label_list if c <= prev_k
                    for r in remaining_rows])
        # Find the first c which have non-zero value among remaining rows
        pivot, k = find_pivot(remaining_rows, col_label_list, prev_k)
        remaining_rows.remove(pivot)

        return [pivot] + _convert_in_echelon(remaining_rows, k)

    remaining_rows = list(range(len(rowlist)))
    prev_k = -1

    return [rowlist[r] for r in _convert_in_echelon(remaining_rows, prev_k)]

rowlist = [
    [0, 2, 3, 4, 5],
    [0, 0, 0, 3, 2],
    [1, 2, 3, 4, 5],
    [0, 0, 0, 6, 7],
    [0, 0, 0, 9, 9]
]
rowlist2 = [
    [0, 0, 0, 0, 0, 9],
    [0, 0, 1, 0, 3, 4],
    [0, 0, 0, 0, 1, 2],
    [0, 2, 3, 0, 5, 6],
]

# [
#     convert_in_echelon(rowlist, list(range(5))),
#     convert_in_echelon(rowlist2, list(range(6)))
# ]

In [3]:
from book.vecutil import list2vec
from ch5_problem import accept_list

rowlist = [
    [0, 2, 3, 4, 5],
    [0, 0, 0, 3, 2],
    [1, 2, 3, 4, 5],
    [0, 0, 0, 6, 7],
    [0, 0, 0, 9, 8]
]

@accept_list
def row_reduce(rowlist):
    col_label_list = sorted(rowlist[0].D, key=hash)
    rows_left = set(range(len(rowlist)))
    new_rowlist = []

    for c in col_label_list:
        rows_with_nonzero = [r for r in rows_left if rowlist[r][c] != 0]
        if rows_with_nonzero:
            pivot = rows_with_nonzero[0]
            new_rowlist.append(rowlist[pivot])
            rows_left.remove(pivot)
            # to eliminate the non-zero entries for remaining rows at column pivot
            for r in rows_with_nonzero[1:]:
                multiplier = rowlist[r][c]/rowlist[pivot][c]
                rowlist[r] -= multiplier * rowlist[pivot]
    return new_rowlist

row_reduce(rowlist)

[Vec({0, 1, 2, 3, 4},{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 3, 4: 2}),
 Vec({0, 1, 2, 3, 4},{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 3.0})]

In [4]:
from book.GF2 import one

gf_list = [
    [0, 0, one, one],
    [one, 0, one, one],
    [one, 0, 0, one],
    [one, one, one, one]
]

row_reduce(gf_list)

[Vec({0, 1, 2, 3},{0: one, 1: 0, 2: one, 3: one}),
 Vec({0, 1, 2, 3},{0: 0, 1: one, 2: 0, 3: 0}),
 Vec({0, 1, 2, 3},{0: 0, 1: 0, 2: one, 3: one}),
 Vec({0, 1, 2, 3},{0: 0, 1: 0, 2: 0, 3: one})]

In [6]:
from book.vec import Vec

@accept_list
def row_reduce(rowlist):
    one = 1
    m = len(rowlist)
    row_labels = set(range(m))
    M_rowlist = [Vec(row_labels, {i:one}) for i in range(m)]
    new_M_rowlist = []

    col_label_list = sorted(rowlist[0].D, key=hash)
    rows_left = set(range(len(rowlist)))
    new_rowlist = []
    M_new_rowlist = []

    for c in col_label_list:
        rows_with_nonzero = [r for r in rows_left if rowlist[r][c] != 0]
        if rows_with_nonzero:
            pivot = rows_with_nonzero[0]
            new_rowlist.append(rowlist[pivot])
            M_new_rowlist.append(M_rowlist[pivot])
            rows_left.remove(pivot)
            # to eliminate the non-zero entries for remaining rows at column pivot
            for r in rows_with_nonzero[1:]:
                multiplier = rowlist[r][c]/rowlist[pivot][c]
                rowlist[r] -= multiplier * rowlist[pivot]
                M_rowlist[r] -= multiplier * M_rowlist[pivot]

    for r in rows_left: M_new_rowlist.append(M_rowlist[r])

    return new_rowlist, M_new_rowlist

row_reduce(rowlist)

([Vec({0, 1, 2, 3, 4},{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}),
  Vec({0, 1, 2, 3, 4},{0: 0, 1: 2, 2: 3, 3: 4, 4: 5}),
  Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 3, 4: 2}),
  Vec({0, 1, 2, 3, 4},{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 3.0})],
 [Vec({0, 1, 2, 3, 4},{2: 1}),
  Vec({0, 1, 2, 3, 4},{0: 1}),
  Vec({0, 1, 2, 3, 4},{1: 1}),
  Vec({0, 1, 2, 3, 4},{0: 0.0, 1: -2.0, 2: 0.0, 3: 1.0, 4: 0.0}),
  Vec({0, 1, 2, 3, 4},{0: 0.0, 1: -1.6666666666666667, 2: 0.0, 3: -0.6666666666666666, 4: 1.0})])